In [1]:
# Install Libraries (This may need to be done first each time the notebook is used here.  Takes a few minutes to install)
from IPython.display import clear_output
try:
  !pip install pystan
  !pip install --upgrade git+https://github.com/jroakes/google-analytics.git
  !pip install fbprophet
except:
  pass
finally:
  clear_output()
  print('All Loaded')

All Loaded


In [95]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


import numpy as np
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot

In [126]:
# load data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = pd.read_csv(path, header=0)
# summarize shape
print(df.shape)
# show first few rows
print(df.head())

(108, 2)
     Month  Sales
0  1960-01   6550
1  1960-02   8728
2  1960-03  12026
3  1960-04  14395
4  1960-05  14587


In [127]:
df["agencia"] = "A"

In [128]:
for i in ["b", "c", "D", "E", "f", "g", "H"]:
  print (i)
  df_aux = df.query("agencia == 'A'").copy()
  df_aux["agencia"] = i
  df_aux["Sales"] = df_aux["Sales"] * np.random.randint(1,100) /10

  df = pd.concat([df,df_aux])


b
c
D
E
f
g
H


PRophet

In [129]:
df["data"] = pd.to_datetime(df["Month"]) 

In [131]:
df_treino = df.query("data < '1966-01-01'")
df_teste = df.query("data  >= '1966-01-01'")

In [130]:
future = pd.DataFrame(df_teste["data"].unique())
future.columns = ['ds']

In [132]:
df_treino.head()

,Month,Sales,agencia,data
0,1960-01,6550.0,A,1960-01-01
1,1960-02,8728.0,A,1960-02-01
2,1960-03,12026.0,A,1960-03-01
3,1960-04,14395.0,A,1960-04-01
4,1960-05,14587.0,A,1960-05-01


In [133]:
df_teste["agencia"].value_counts()

c    36
A    36
E    36
H    36
f    36
b    36
D    36
g    36
Name: agencia, dtype: int64

In [152]:
# criando o dataset de futuro - que será o nosso de validação
future = pd.DataFrame(df_teste["data"].unique())
future.columns = ['ds']

# Treinando para cada agencia


# cria o dataframe com as predições para cada agencia
yhats = pd.DataFrame()

# treinando um modelo para cada agencia
for i in df_treino["agencia"].unique():

  df_prophet = df_treino.query(f"agencia == '{i}'")[["data", "Sales"]]
  df_prophet = df_prophet.rename(columns={"data" : "ds", "Sales" : "y"})
  
  prophet = Prophet()
  prophet.add_country_holidays(country_name='US')
  prophet.fit(df_prophet)
  forecast = prophet.predict(future)

# preenchendo o dataframe com as preedições
  aux = pd.DataFrame()
  string = f'{i}'
  aux["ds"] = forecast["ds"]
  aux["yhat"] = forecast["yhat"]
  aux["agencia"] = string
  
  yhats = pd.concat([yhats, aux])


  # fig = prophet.plot(forecast)
  # a = add_changepoints_to_plot(fig.gca(), prophet, forecast)


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [154]:
df_teste["ds"] = df_teste["data"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [155]:
results = pd.merge(df_teste, yhats, on =['ds', 'agencia'], how="left")
results["erro_absoluto"] = np.abs(results["Sales"] - results["yhat"])

In [157]:
results["erro_absoluto"] = np.abs(results["Sales"] - results["yhat"])
results["erro_absoluto"].mean()

9319.720775188129

In [153]:
results["erro_absoluto"].mean()

9315.826416385864

# parameteres tunning

In [159]:
df_treino = df.query("data < '1966-01-01'")
df_teste = df.query("data  >= '1966-01-01'")

In [171]:
from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200]}
params_grid = {'seasonality_mode':('multiplicative','additive')}


grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 2


In [190]:
# criando o dataset de futuro - que será o nosso de validação
future = pd.DataFrame(df_teste["data"].unique())
future.columns = ['ds']

# Treinando para cada agencia

MAE_dict = {}

# cria o dataframe com as predições para cada agencia
yhats = pd.DataFrame()

# treinando um modelo para cada agencia
for i in df_treino["agencia"].unique():

  df_prophet = df_treino.query(f"agencia == '{i}'")[["data", "Sales"]]
  df_prophet = df_prophet.rename(columns={"data" : "ds", "Sales" : "y"})

  model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
  
  for p in grid:
    print(i,p)
    prophet = Prophet(#changepoint_prior_scale = p['changepoint_prior_scale'],
                       # holidays_prior_scale = p['holidays_prior_scale'],
                       # n_changepoints = p['n_changepoints'],
                        seasonality_mode = p['seasonality_mode'],
                        weekly_seasonality=True,
                        daily_seasonality = True,
                        yearly_seasonality = True, 
                        interval_width=0.95)
    prophet.fit(df_prophet)
    forecast = prophet.predict(future)

    df_teste_aux = df_teste.query(f"agencia == '{i}'")[["data", "Sales"]]
    df_teste_aux = df_teste_aux.rename(columns={"data" : "ds", "Sales" : "y"}) 

    results = pd.merge(df_teste_aux, forecast[["ds", "yhat"]], on="ds")
    results["erro_absoluto"] = np.abs(results["y"] - results["yhat"])
    MAE = results["erro_absoluto"].mean()

    model_parameters = model_parameters.append({'MAPE':MAE,'Parameters':p},ignore_index=True)

    MAE_dict[i] = model_parameters



A {'seasonality_mode': 'multiplicative'}
A {'seasonality_mode': 'additive'}
b {'seasonality_mode': 'multiplicative'}
b {'seasonality_mode': 'additive'}
c {'seasonality_mode': 'multiplicative'}
c {'seasonality_mode': 'additive'}
D {'seasonality_mode': 'multiplicative'}
D {'seasonality_mode': 'additive'}
E {'seasonality_mode': 'multiplicative'}
E {'seasonality_mode': 'additive'}
f {'seasonality_mode': 'multiplicative'}
f {'seasonality_mode': 'additive'}
g {'seasonality_mode': 'multiplicative'}
g {'seasonality_mode': 'additive'}
H {'seasonality_mode': 'multiplicative'}
H {'seasonality_mode': 'additive'}


In [193]:
for i in df_treino["agencia"].unique():
  print(MAE_dict[i].sort_values("MAPE", ascending=True)[:1])

          MAPE                        Parameters
1  2547.381775  {'seasonality_mode': 'additive'}
          MAPE                        Parameters
1  6623.192616  {'seasonality_mode': 'additive'}
         MAPE                        Parameters
1  5094.76355  {'seasonality_mode': 'additive'}
           MAPE                        Parameters
1  12227.432521  {'seasonality_mode': 'additive'}
         MAPE                        Parameters
1  3056.85813  {'seasonality_mode': 'additive'}
         MAPE                        Parameters
1  3056.85813  {'seasonality_mode': 'additive'}
           MAPE                        Parameters
1  21398.006912  {'seasonality_mode': 'additive'}
           MAPE                        Parameters
1  16303.243361  {'seasonality_mode': 'additive'}
